In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#log_files_path = '/content/drive/My Drive/EIB project/'
import os 
import sys
sys.path.insert(0, os.path.abspath('./modules'))
import glob
import pandas as pd
#import features
import numpy as np
import collections
import random
import matplotlib.pyplot as plt
import re
from sklearn import metrics
from sklearn.model_selection import train_test_split
import nltk
import datetime
#from textblob import TextBlob as tb
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import SnowballStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
import inflect
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
from sklearn import utils as skl_utils
from tqdm import tqdm
import xgboost as xgb
import multiprocessing
from gensim import utils
import gensim.parsing.preprocessing as gsp
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Prepare Data

In [ ]:
score=pd.read_csv('real BB company.csv')
score.drop(columns=['Unnamed: 0'],inplace=True)
score.rename(columns={"Name":"company"},inplace=True)
corpus=pd.read_csv('real whole corpus.csv',names=['company','text'])
data=pd.merge(corpus, score, on=['company'], how='inner').drop_duplicates('company').reset_index(drop=True)
data.dropna(inplace=True)
x_feature=data.values[:,1]
y=data.values[:,-1]

## Build Model

In [ ]:
class Doc2VecTransformer(BaseEstimator):

    def __init__(self, vector_size=100, learning_rate=0.02, epochs=20):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self._model = None
        self.vector_size = vector_size
        self.workers = multiprocessing.cpu_count() - 1


    def clean_text(self,s):
        filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]
        s = s.lower()
        s = utils.to_unicode(s)
        for f in filters:
            s = f(s)
        return s
    
    def fit(self, df_x, df_y=None):
        tagged_x = [TaggedDocument(self.clean_text(row).split(), [index]) for index, row in enumerate(df_x)]
        model = Doc2Vec(documents=tagged_x, vector_size=self.vector_size, workers=self.workers)

        for epoch in range(self.epochs):
            model.train(skl_utils.shuffle([x for x in tqdm(tagged_x)]), total_examples=len(tagged_x), epochs=1)
            model.alpha -= self.learning_rate
            model.min_alpha = model.alpha

        self._model = model
        return self

    def transform(self, df_x):
        return np.asmatrix(np.array([self._model.infer_vector(self.clean_text(row).split())
                                     for index, row in enumerate(df_x)]))

In [ ]:
class Doc2vecXGB:
    def select_model(self,x,y,nthread,objective,learning_rate,max_depth,
                     min_child_weight,subsample,colsample_bytree,n_estimators):
        parameters = {'nthread':nthread, #when use hyperthread, xgboost may become slower
              'objective':objective,
              'learning_rate': learning_rate, #so called `eta` value
              'max_depth': max_depth,
              'min_child_weight': min_child_weight,
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'n_estimators': n_estimators}
        
        xgb1 = XGBRegressor()
        xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)
        xgb_grid.fit(x,y)
        print("best score:",xgb_grid.best_score_)
        print("best params:",xgb_grid.best_params_)
        self.best_model=xgb_grid.best_estimator_
        
    
    def train(self,x,y):
        self.best_model.fit(x,y)
    
    def predict(self,x):
        return self.best_model.predict(x)

        

    

## Train model

In [ ]:
## generate doc2vec feature
doc2vec_trf = Doc2VecTransformer()
doc2vec_features = doc2vec_trf.fit(x_feature).transform(x_feature)

##train test split
trainX,testX,trainY,testY=train_test_split(doc2vec_features,y,test_size=0.2,shuffle=True)

100%|██████████| 546/546 [00:00<00:00, 1499731.49it/s]


In [ ]:
# Model Training
model=Doc2vecXGB()
## grid search
model.select_model(trainX,trainY,nthread=[4],objective=['reg:squarederror'],learning_rate=[.03, 0.05, .07],max_depth=[5, 6, 7],
                     min_child_weight=[4],subsample=[0.7],colsample_bytree=[0.7],n_estimators=[500])
model.train(trainX,trainY)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  18 out of  18 | elapsed:   32.9s finished


best score: 0.3148119367267067
best params: {'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:squarederror', 'subsample': 0.7}


In [ ]:
#training MSE
MSE = np.mean((model.predict(trainX) - trainY)**2)
print(MSE)

2.4185648448157417


In [ ]:
# Testing MSE:
MSE1 = np.mean((model.predict(testX)- testY)**2)
print(MSE1)

275.22245557317905
